In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/commonlit-evaluate-student-summaries/sample_submission.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import SparsePCA,PCA,KernelPCA

nltk.download('wordnet')
nltk.download('stopwords')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


False

In [3]:
summary_df = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
prompt_df = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')

In [4]:
def clean_text(text):
    stops = stopwords.words('english')
    processed_text = []
    for word in text.split():
        if word not in stops:
            processed_text.append(word)
    return ' '.join(processed_text)

def stem_word(text):
    stem = LancasterStemmer()
    stemmed_data = []
    for word in text.split():
        stemmed_data.append(stem.stem(word.lower()))
    return ' '.join(stemmed_data)

In [5]:
def preprocess_data(summary_df,prompt_df,ngram_range=(1,1)):
    train_df = summary_df.merge(right=prompt_df,how='inner', on='prompt_id')
    train_df['full_text'] = train_df.apply(lambda x: 'Title:\n'+x['prompt_title']+
                   '\nQuestion:\n'+x['prompt_question']+
                   '\nText:\n'+x['prompt_text']+
                   '\nAnswer:\n'+x['text'],axis=1)
    train_df['full_text'] = train_df['full_text'].apply(clean_text)
    train_df['full_text'] = train_df['full_text'].apply(stem_word)
    train_df['length_of_summary'] = train_df['text'].apply(lambda x: len(x))
    train_df['summary_text'] = train_df.apply(lambda x: 'Question:\n'+x['prompt_question']+'\nAnswer:\n'+x['text'],axis=1)
    df = train_df[['student_id','full_text','summary_text','length_of_summary','content','wording']].copy()
    global transformer, vectorizer, scaler, pca
    vectorizer = CountVectorizer(ngram_range=ngram_range)
    transformer = TfidfTransformer()
    transformed_df = transformer.fit_transform(vectorizer.fit_transform(df['full_text']))
    summary_transformed_df = transformer.transform(vectorizer.transform(df['summary_text']))
    scaler = StandardScaler(with_mean=True, with_std=True,)
    X_sample_scaled = scaler.fit_transform(transformed_df.toarray())
    X_summary_scaled = scaler.fit_transform(summary_transformed_df.toarray())
    pca = PCA(n_components=2207)
    pca_bag = pca.fit_transform(X_sample_scaled)
    summary_bag = pca.transform(X_summary_scaled)
    stacked_bag = np.hstack((pca_bag,summary_bag,df[['length_of_summary','content','wording']].values))
    student_ids = df['student_id'].values
    return stacked_bag, student_ids

In [6]:
stacked_bag, student_ids = preprocess_data(summary_df,prompt_df,ngram_range=(1,1))

In [7]:
X_train,X_test,y_train,y_test = train_test_split(stacked_bag[:,:-2] ,stacked_bag[:,-2:],test_size=0.1,shuffle=True,random_state=11)

In [8]:
def mcrmse(y_true, y_pred):
    return np.mean(np.sqrt(np.mean((y_true - y_pred)**2, axis=0)))

mcrmse_scorer = make_scorer(mcrmse, greater_is_better=False)
model_rf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100,criterion='squared_error',max_depth=5,random_state=11),n_jobs=-1)

In [9]:
model_rf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100,criterion='squared_error',max_depth=5,random_state=11),n_jobs=-1)

In [10]:
model_rf.fit(X_train,y_train)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=5,
                                                     random_state=11),
                     n_jobs=-1)

In [11]:
y_preds = model_rf.predict(X_test)

In [12]:
mcrmse(y_test,y_preds)

0.558861869185053

In [13]:
model_rf.fit(stacked_bag[:,:-2], stacked_bag[:,-2:])

MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=5,
                                                     random_state=11),
                     n_jobs=-1)

In [14]:
vectorizer

CountVectorizer()

In [15]:
scaler

StandardScaler()

In [16]:
def preprocess_data(summary_df, prompt_df, vectorizer=vectorizer,scaler=scaler,transformer=transformer,pca=pca):
    train_df = summary_df.merge(right=prompt_df,how='inner', on='prompt_id')
    train_df['full_text'] = train_df.apply(lambda x: 'Title:\n'+x['prompt_title']+
                   '\nQuestion:\n'+x['prompt_question']+
                   '\nText:\n'+x['prompt_text']+
                   '\nAnswer:\n'+x['text'],axis=1)
    train_df['full_text'] = train_df['full_text'].apply(clean_text)
    train_df['full_text'] = train_df['full_text'].apply(stem_word)
    train_df['length_of_summary'] = train_df['text'].apply(lambda x: len(x))
    train_df['summary_text'] = train_df.apply(lambda x: 'Question:\n'+x['prompt_question']+'\nAnswer:\n'+x['text'],axis=1)
    df = train_df[['student_id','full_text','summary_text','length_of_summary']].copy()
    transformed_df = transformer.transform(vectorizer.transform(df['full_text']))
    summary_transformed_df = transformer.transform(vectorizer.transform(df['summary_text']))
    X_sample_scaled = scaler.transform(transformed_df.toarray())
    X_summary_scaled = scaler.transform(summary_transformed_df.toarray())
    pca_bag = pca.transform(X_sample_scaled)
    summary_bag = pca.transform(X_summary_scaled)
    stacked_bag = np.hstack((pca_bag,summary_bag,df['length_of_summary'].values.reshape(-1,1)))
    student_ids = df['student_id'].values
    return stacked_bag, student_ids

In [17]:
test_summary = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')
test_prompts = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')

In [18]:
test_df,ids = preprocess_data(test_summary, test_prompts, vectorizer=vectorizer,scaler=scaler,transformer=transformer,pca=pca)

In [19]:
y_preds = model_rf.predict(test_df)

In [20]:
df_test = pd.DataFrame(y_preds,columns=['content','wording'],index=ids).reset_index()
df_test = df_test.rename({'index':'student_id'},axis=1)

In [21]:
df_test.to_csv("submission.csv", index=False)